In [1]:
from PIL import Image, ImageEnhance
import numpy as np 
import os
import cv2
from skimage.color import rgb2lab, lab2rgb
from skimage import color
from skimage.io import imsave
# np.save('example_1', a)
# b = np.load('example/example_1.npy')
# from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
# from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
# from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
import pickle
# import tensorflow as tf
# from keras.models import load_model
# import keras 

In [78]:
def transform_image(img_pil,brightness,contrast,color):
    if (brightness != 0):
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(brightness)
    if (contrast != 0):
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(contrast)
    if (color != 0):
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(color) 
    return img_pil
array_transform_nastr = [
    [1,1,0.8]
]

# Заполнение идеальных данных

In [32]:
def crop_image(data,a,b):
    h=0
    while h <= a.shape[0]-pixel:
        w=0
        while w <= a.shape[1]-pixel:
            data.append([a[h,w][1],a[h,w][2],b[h,w][1],b[h,w][2]])
            w+=pixel
        h+=pixel

In [95]:
# ИЗ РАЗНЫХ ПАПОК
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/type"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for dir_ in dirs:
        #print (dir_)
        for root_, dirs_, files_ in os.walk(image_folder + "/" + dir_):
            for file in files_:
                if file == ".DS_Store" or not file[0] == "_" or file[1] == "Y":
                    continue
                if file[1] == "X":
                    #print(file)
                    img_X = Image.open(root_ + "/" + file)
                    img_X = img_X.convert('RGB')
                    img_X_lab = rgb2lab(img_X)
                    
                    img_Y = Image.open(root_ + "/_Y.jpeg")
                    img_Y = img_Y.convert('RGB')
                    img_Y_lab = rgb2lab(img_Y)

                    array_image.append([np.array(img_X_lab),np.array(img_Y_lab)])
                    
array_full_image = np.array(array_image) 
array_full_image.shape

/var/folders/87/c02w9bp14k1fx5943h90k0500000gn/T/ipykernel_57010/820985584.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_full_image = np.array(array_image)


(25, 2)

In [87]:
# ИЗ ОДНОЙ ПАПКИ

image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/Portra"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file == ".DS_Store":
            continue
        img_pil = Image.open(root + "/" + file)
        img_pil = img_pil.convert('RGB')
        img_pil_lab = rgb2lab(img_pil)
        #img_cv2_ish = cv2.imread(root + "/" + file)
        for x in array_transform_nastr:
            img_ = transform_image(img_pil,x[0],x[1],x[2])
            #img_.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            #img_cv2_transf = cv2.imread("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            img_lab = rgb2lab(img_)
            #img_pil = rgb2lab(img_pil)
            array_image.append([np.array(img_lab),np.array(img_pil_lab)])
        
array_full_image = np.array(array_image) 
array_full_image.shape

/var/folders/87/c02w9bp14k1fx5943h90k0500000gn/T/ipykernel_57010/336729453.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_full_image = np.array(array_image)


(9, 2)

In [96]:
data = []
i=0
x_sgape = array_full_image.shape[0]
while i < x_sgape:
    crop_image(data,array_full_image[i,0],array_full_image[i,1])
    i+=1

dFrame = pd.DataFrame(data,columns=['X_a','X_b','Y_a','Y_b']) 

Slide_ =  dFrame[['Y_a', 'Y_b']]
Slide_ = Slide_.drop_duplicates()

Slide_.loc[:, 'type'] = "slide"


In [97]:
Slide_

,Y_a,Y_b,type
0,19.895634,-37.474080,slide
1,18.785200,-37.375162,slide
2,18.265667,-36.413036,slide
3,16.763190,-35.076124,slide
5,16.492175,-35.840877,slide
...,...,...,...
22985943,8.917132,-29.625542,slide
22985944,8.051799,-27.791456,slide
22986328,6.655849,-20.935637,slide
22986719,1.683488,-20.131695,slide


# Подготовка необработанных данных

In [53]:
temp_folder = "/Users/alexeyvaganov/doc/files/color_matcher/temp/"

In [54]:
file_name_massiv = []
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_3379.jpeg")
file_name_massiv.append("IMG_2915.jpeg")
file_name_massiv.append("IMG_2980.jpeg")
file_name_massiv.append("IMG_3107.jpeg")
file_name_massiv.append("IMG_3323.jpeg")
file_name_massiv.append("IMG_3336.jpeg")
file_name_massiv.append("IMG_2769.jpeg")
file_name_massiv.append("IMG_2837.jpeg")
file_name_massiv.append("IMG_2847.jpeg")

In [55]:
def add_testing_file(file_name):
    test_image_path = temp_folder + "/" + file_name
    img_test = Image.open(test_image_path)
    img_test = img_test.convert('RGB')
    img_test = rgb2lab(img_test)
    return np.array(img_test) 

In [107]:
def return_number_sovpad(ideal, pref):
    arrive_testing_file = []
    for name_test_file in file_name_massiv:
        arr_ = add_testing_file(pref + name_test_file)
        arrive_testing_file.append([pref  + name_test_file, arr_])
    
    Dig_p = []
    for m_image in arrive_testing_file:
        h=0
        while h <= m_image[1].shape[0]-pixel:
            w=0
            while w <= m_image[1].shape[1]-pixel:
                Dig_p.append([m_image[1][h,w][1],m_image[1][h,w][2]])
                w+=pixel
            h+=pixel 

    Digital_ = pd.DataFrame(Dig_p,columns=['Y_a','Y_b']) 
    Digital_ = Digital_.drop_duplicates()
    Digital_.loc[:, 'type'] = "digital"
    
    bigdata = pd.concat([ideal,Digital_],ignore_index=True)
    
    groupideal = ideal.groupby(['Y_a', 'Y_b']).ngroups
    groupDigital = Digital_.groupby(['Y_a', 'Y_b']).ngroups
    group = bigdata.groupby(['Y_a', 'Y_b']).ngroups
    
    print ("for " + str(pref))
    print ("groupideal " + str(groupideal))
    print ("groupDigital " + str(groupDigital))
    print ("group " + str(group))
    
    return groupideal + groupDigital - group

In [108]:
number_S_0_ = return_number_sovpad(Slide_,"")
number_S_57_ = return_number_sovpad(Slide_,"57_")
number_S_60_ = return_number_sovpad(Slide_,"60_")
number_S_61_ = return_number_sovpad(Slide_,"61_")
number_S_66_ = return_number_sovpad(Slide_,"66_")
number_S_67_ = return_number_sovpad(Slide_,"67_")
number_S_68_ = return_number_sovpad(Slide_,"68_")
number_S_81_ = return_number_sovpad(Slide_,"81_")
number_S_89_ = return_number_sovpad(Slide_,"89_")
number_S_92_ = return_number_sovpad(Slide_,"92_")
number_S_93_ = return_number_sovpad(Slide_,"93_")
number_S_94_ = return_number_sovpad(Slide_,"94_")
number_S_95_ = return_number_sovpad(Slide_,"95_")

for 
groupideal 1066293
groupDigital 560948
group 1228118
for 57_
groupideal 1066293
groupDigital 330425
group 1115366
for 60_
groupideal 1066293
groupDigital 440190
group 1150106
for 61_
groupideal 1066293
groupDigital 439077
group 1157022
for 66_
groupideal 1066293
groupDigital 358929
group 1120064
for 67_
groupideal 1066293
groupDigital 416911
group 1132180
for 68_
groupideal 1066293
groupDigital 298279
group 1114458
for 81_
groupideal 1066293
groupDigital 476568
group 1160059
for 89_
groupideal 1066293
groupDigital 539302
group 1191335
for 92_
groupideal 1066293
groupDigital 357430
group 1182442
for 93_
groupideal 1066293
groupDigital 555208
group 1168912
for 94_
groupideal 1066293
groupDigital 538957
group 1168514
for 95_
groupideal 1066293
groupDigital 535803
group 1163114


In [104]:
number_S_0_ = return_number_sovpad(Slide_,"")

groupideal1066293
groupDigital560948
group1228118


In [85]:
number_S_57_ = return_number_sovpad(Slide_,"57_")

In [66]:
number_S_61_ = return_number_sovpad(Slide_,"61_")

In [67]:
number_S_66_ = return_number_sovpad(Slide_,"66_")

In [68]:
number_S_67_ = return_number_sovpad(Slide_,"67_")

In [69]:
number_S_68_ = return_number_sovpad(Slide_,"68_")

In [70]:
number_S_81_ = return_number_sovpad(Slide_,"81_")

In [71]:
number_S_89_ = return_number_sovpad(Slide_,"89_")

In [72]:
number_S_92_ = return_number_sovpad(Slide_,"92_")

In [74]:
number_S_93_ = return_number_sovpad(Slide_,"93_")

In [75]:
number_S_94_ = return_number_sovpad(Slide_,"94_")

In [76]:
number_S_95_ = return_number_sovpad(Slide_,"95_")

In [109]:
print("S0 - " + str(number_S_0_))
print("S57 - " + str(number_S_57_))
print("S60 - " + str(number_S_60_))
print("S61 - " + str(number_S_61_))
print("S66 - " + str(number_S_66_))
print("S67 - " + str(number_S_67_))
print("S68 - " + str(number_S_68_))
print("S81 - " + str(number_S_81_))
print("S89 - " + str(number_S_89_))
print("S92 - " + str(number_S_92_))
print("S93 - " + str(number_S_93_))
print("S94 - " + str(number_S_94_))
print("S95 - " + str(number_S_95_))

S0 - 399123
S57 - 281352
S60 - 356377
S61 - 348348
S66 - 305158
S67 - 351024
S68 - 250114
S81 - 382802
S89 - 414260
S92 - 241281
S93 - 452589
S94 - 436736
S95 - 438982


In [110]:
my_image= Image.open("/Users/alexeyvaganov/doc/files/color_matcher/temp/IMG_2847.jpeg")
my_image = my_image.convert('RGB')
my_image_lab = rgb2lab(my_image)
my_image_lab.shape

(1280, 960, 3)

In [111]:
my_image_lab[0,0]

array([ 15.07521656, -12.67294754,  18.01043186])